In [1]:
import openai
import pandas as pd

gpt설정

In [2]:
def question(query):

    # FTA API
    API_KEY = ""
    openai.api_key = API_KEY

    model = "gpt-3.5-turbo"  

    content = """
You are a binary classifier.
When programming code comes in, you perform binary classification according to the given rules.
Classify as 1 if it meets the rule, otherwise classify as 0.
You can only answer about binary classification results. The results are in the form of a list.
"""

    messages = [
        {"role":"system", "content": content},
        {"role":"user","content":query}
    ]

    response = openai.ChatCompletion.create(
        temperature=0.8,
        model=model,
        messages = messages
    )

    answer = response['choices'][0]['message']['content']

    return answer


In [53]:
# CSV 파일 읽기
df = pd.read_excel("../1차결과.xlsx") 
columns_list = df.columns.tolist()

df

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
0,강지민 jiminkang1404,22.11.01 22:08,10보다작을까?-1,AC,NaN,#include <cstdio>\nint main(){\n int a;\n ...,NOT,1,NaN
1,권현욱,21.12.26 10:24,10보다작을까?-1,AC,NaN,#include<stdio.h>\nint main()\n{\nint a;\nscan...,NOT,1,NaN
2,김강범 kkb110513,22.07.30 18:11,10보다작을까?-1,AC,NaN,#include <cstdio>\n\nint main()\n{\n int a;...,NOT,1,NaN
3,김동우 derek0602,22.11.05 10:41,10보다작을까?-1,AC,NaN,#include<cstdio>\nint main(){\n int a;\n ...,NOT,1,NaN
4,김민하,21.12.18 11:24,10보다작을까?-1,AC,NaN,"#include<cstdio>\nint main(){\nint a;\nscanf(""...",NOT,1,NaN
...,...,...,...,...,...,...,...,...,...
1404,허윤구,21.03.07 11:15,사칙연산,AC,NaN,"#include <cstdio>\nint main()\n{\n int a, b...",NOT,2,NaN
1405,허윤구,21.03.07 11:11,사칙연산,WA,-1,"#include <cstdio>\nint main(){\n int a, b;\...",NOT,2,NaN
1406,허윤구,21.03.07 11:10,사칙연산,WA,-1,"#include <cstdio>\nint main()\n{\n int a, b...",NOT,4,NaN
1407,허윤구,21.03.07 11:07,사칙연산,WA,"0,2","#include<cstdio>\nint main()\n{\n int a, b;...","0,0",0,NaN


In [54]:
df["오류 분류"] = df["오류 분류"].str.replace(r"\s+", "", regex=True)

split_error_df = df["오류 분류"].str.split(",", expand=True).stack().reset_index(level=1, drop=True).rename("오류 분류")

split_df = df.drop("오류 분류", axis=1).join(split_error_df).reset_index(drop=True)


split_df = split_df[columns_list]

split_df

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
0,강지민 jiminkang1404,22.11.01 22:08,10보다작을까?-1,AC,NaN,#include <cstdio>\nint main(){\n int a;\n ...,NOT,1,NaN
1,권현욱,21.12.26 10:24,10보다작을까?-1,AC,NaN,#include<stdio.h>\nint main()\n{\nint a;\nscan...,NOT,1,NaN
2,김강범 kkb110513,22.07.30 18:11,10보다작을까?-1,AC,NaN,#include <cstdio>\n\nint main()\n{\n int a;...,NOT,1,NaN
3,김동우 derek0602,22.11.05 10:41,10보다작을까?-1,AC,NaN,#include<cstdio>\nint main(){\n int a;\n ...,NOT,1,NaN
4,김민하,21.12.18 11:24,10보다작을까?-1,AC,NaN,"#include<cstdio>\nint main(){\nint a;\nscanf(""...",NOT,1,NaN
...,...,...,...,...,...,...,...,...,...
1466,허윤구,21.03.07 11:11,사칙연산,WA,-1,"#include <cstdio>\nint main(){\n int a, b;\...",NOT,2,NaN
1467,허윤구,21.03.07 11:10,사칙연산,WA,-1,"#include <cstdio>\nint main()\n{\n int a, b...",NOT,4,NaN
1468,허윤구,21.03.07 11:07,사칙연산,WA,0,"#include<cstdio>\nint main()\n{\n int a, b;...","0,0",0,NaN
1469,허윤구,21.03.07 11:07,사칙연산,WA,2,"#include<cstdio>\nint main()\n{\n int a, b;...","0,0",0,NaN


매핑 규칙

In [65]:
mapping = {
    "10보다작을까?-1" :  { 
    "0": "Input format mismatch",
    "1": "Incorrect syntax",
    "2": "off-by-one",
    "3": "Output format mismatch"
    },

    "10보다작을까?-2" : {
    '0': 'off-by-one',
    '1' : 'Output format mismatch',
    '2' : 'Input format mismatch',
    '3' : 'Output format mismatch',
    '4' : 'Incorrect syntax'
    },

    "내나이가어때서" : {
    '0' : 'Incorrect calculation',
    '1' : 'Incorrect calculation',
    '2' : 'Output format mismatch',
    '3' : 'Incorrect calculation',
    '4' : 'Input format mismatch'
    },

    "두수비교하기(<=)" : {
    '0' : 'Output format mismatch',
    '1' : 'Output format mismatch',
    '2' : 'Incorrect syntax',
    '3' : 'Incorrect calculation'
    },

    "두수비교하기(==)" : {
    '0' : 'Output format mismatch',
    '1' : 'Output format mismatch',
    '2' : 'Incorrect syntax',
    '3' : 'Incorrect calculation',
    '4' : 'Input format mismatch'
    }, 

    "두정수바꾸어출력하기" : {
    '0' : 'misunderstanding',
    '1' : 'Incorrect calculation',
    '2' : 'Output format mismatch',
    '3' : 'Input format mismatch',
    '4' : 'Output format mismatch'
    },

    "두정수입력받기" : {
    '0' : 'Output format mismatch',
    '1' : 'Output format mismatch',
    '2' : 'Output format mismatch',
    '3' : 'Incorrect syntax',
    '4' : 'Input format mismatch',
    '5' : 'Input format mismatch',
    '6' : 'Incorrect calculation',
    '7' : 'Input format mismatch'
    },

    "반값이벤트" : {
    '0' : 'Output format mismatch',
    '1' : 'Incorrect calculation',
    '2' : 'Output format mismatch',
    '3' : 'Input format mismatch'
    },

    "사칙연산" : {
    '0' : 'Output format mismatch',
    '1' : 'Output format mismatch',
    '2' : 'Output format mismatch',
    '3' : 'Incorrect calculation',
    '4' : 'Output format mismatch'
    }, 

    "시간단위로변환하기" : {
    '0' : 'Incorrect calculation',
    '1' : 'Incorrect calculation',
    '2' : 'Input format mismatch',
    '3' : 'Output format mismatch'
    }, 

    "원의넓이구하기" : {
    '0' : 'type mismatch',
    '1' : 'type mismatch',
    '2' : 'type mismatch',
    '3' : 'Output format mismatch',
    '4' : 'Incorrect calculation'
    }, 

    "음수는0으로" : {
    '0' : 'Output format mismatch',
    '1' : 'Incorrect calculation',
    '2' : 'Incorrect calculation',
    '3' : 'type mismatch',
    '4' : 'Incorrect calculation'
    },

    "정수한개입력받아부호바꿔출력하기" : {
    '0' : 'Incorrect calculation',
    '1' : 'Output format mismatch',
    '2' : 'Input format mismatch'
    },

    "초를분,초단위로변환하기" : {
    '0' : 'Output format mismatch',
    '1' : 'Output format mismatch',
    '2' : 'Output format mismatch',
    '3' : 'Incorrect calculation',
    '4' : 'Input format mismatch'
    }
     

    
}

In [66]:
# 오류 분류 매핑 함수
def map_error_classification(row):
    problem = row['문제 이름']
    error_code = str(row['오류 분류'])
    
    if problem in mapping and error_code in mapping[problem]:
        return mapping[problem][error_code]
    
    return 'Unknown Error'

# 오류 분류 매핑 적용
split_df['대분류'] = split_df[split_df["정답 여부"] == "WA"].apply(map_error_classification, axis=1)

split_df[split_df["정답 여부"] == "WA"]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
19,유재학 yujaehak16,22.08.07 18:16,10보다작을까?-1,WA,2,#include <cstdio>\nint main()\n{\n int a;\n...,off-by-one,2,NaN
20,유재학 yujaehak16,22.08.07 18:16,10보다작을까?-1,WA,2,#include <cstdio>\nint main()\n{\n int a;\n...,off-by-one,2,NaN
22,유제연,22.02.12 19:45,10보다작을까?-1,WA,NaN,#include<cstdio>\nint main()\n{\nint a;\nif(10...,Unknown Error,1,NaN
26,이연준,20.11.01 18:28,10보다작을까?-1,WA,1,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,Incorrect syntax,1,NaN
32,전현준,22.03.11 17:21,10보다작을까?-1,WA,3,#include <cstdio>\n\nint main()\n{\nint a=11;\...,Output format mismatch,0,NaN
...,...,...,...,...,...,...,...,...,...
1464,허윤구,21.03.07 11:06,사칙연산,WA,2,"#include<cstdio>\nint main()\n{\n int a, b;...",Output format mismatch,0,NaN
1466,허윤구,21.03.07 11:11,사칙연산,WA,-1,"#include <cstdio>\nint main(){\n int a, b;\...",Unknown Error,2,NaN
1467,허윤구,21.03.07 11:10,사칙연산,WA,-1,"#include <cstdio>\nint main()\n{\n int a, b...",Unknown Error,4,NaN
1468,허윤구,21.03.07 11:07,사칙연산,WA,0,"#include<cstdio>\nint main()\n{\n int a, b;...",Output format mismatch,0,NaN


In [67]:
split_df.loc[ (split_df["정답 여부"]=="WA") & (split_df["대분류"] == "Unknown Error"), "대분류"] = "misunderstanding"

In [68]:
unique_values = split_df['대분류'].unique()
print(unique_values)

[nan 'off-by-one' 'misunderstanding' 'Incorrect syntax'
 'Output format mismatch' 'Input format mismatch' 'Incorrect calculation'
 'type mismatch']


In [69]:
split_df.to_excel('../2차결과.xlsx', index=False, engine='openpyxl')

지피티로 한 시도

In [33]:
result_df = split_df

df = split_df.drop(["학생명", "제출 시간", "대분류", "코드", "prediction", "review"], axis=1)
df

,문제 이름,정답 여부,오류 분류
0,10보다작을까?-1,AC,NaN
1,10보다작을까?-1,AC,NaN
2,10보다작을까?-1,AC,NaN
3,10보다작을까?-1,AC,NaN
4,10보다작을까?-1,AC,NaN
...,...,...,...
1466,사칙연산,WA,-1
1467,사칙연산,WA,-1
1468,사칙연산,WA,0
1469,사칙연산,WA,2


In [36]:
df.loc[result_df["정답 여부"] =="WA" ,'대분류'] = result_df.loc[result_df["정답 여부"] =="WA" ,'대분류'].str.lower()

unique_values = df['대분류'].unique()
print(unique_values)

[nan '1' 'not' '0' '0,1' '2' '1,1' '0,0' '2,0' '0,2' '1,0']


10보다 작을까?-1

In [91]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Input format mismatch
1 : semantic error
2 : off-by-one
3 : Ouput format mismatch
NaN : misunderstanding of the problem

"""

result_query = """
2. Answer : 
Please answer in the following example form. The list index matches each code.
Example:
review = [
        "Input format mismatch",
         ...
        ]


"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "10보다작을까?-1") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]].to_string(index=False)

query1_1 = question_query  + result_query + context_query

print(query1_1)


1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Input format mismatch
1 : semantic error
2 : off-by-one
3 : Ouput format mismatch
NaN : misunderstanding of the problem


In cases where there are multiple error classifications for a single entry such as "0,1", 
we can concatenate the error descriptions using an ampersand (&) symbol like "Condition expression error&No input section."


2. Answer : 
Please answer in the following example form. The list index matches each code.
Example:
review = [
        "Condition expression error",
         ...
        ]


The DataFrame contains the following data.

     문제 이름 오류 분류
10보다작을까?-1     2
10보다작을까?-1     2
10보다작을까?-1   NaN
10보다작을까?-1     1
10보다작을까?-1     3
10보다작을까?-1     3
10보다작을까?-1     2
10보다작을까?-1     0
10보다작을까?-1     1
10보다작을까?-1     

In [92]:
ans1_1 = question(query1_1)
print(ans1_1)

Based on the given DataFrame, the error descriptions for each error code are as follows:

review = [
    "off-by-one",
    "off-by-one",
    "misunderstanding of the problem",
    "semantic error",
    "Output format mismatch",
    "Output format mismatch",
    "off-by-one",
    "Input format mismatch",
    "semantic error",
    "semantic error",
    "semantic error",
    "Output format mismatch",
    "Output format mismatch",
    "semantic error",
    "semantic error"
]


In [156]:
review = [
    "off-by-one",
    "off-by-one",
    "misunderstanding of the problem",
    "semantic error",
    "Output format mismatch",
    "Output format mismatch",
    "off-by-one",
    "Input format mismatch",
    "semantic error",
    "semantic error",
    "semantic error",
    "Output format mismatch",
    "Output format mismatch",
    "semantic error",
    "semantic error"
]

ce_rows = result_df[(result_df["문제 이름"] == "10보다작을까?-1") & (result_df["정답 여부"] == "WA")]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "10보다작을까?-1") & (result_df["정답 여부"] == "WA")]


,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
19,유재학 yujaehak16,22.08.07 18:16,10보다작을까?-1,WA,2,#include <cstdio>\nint main()\n{\n int a;\n...,1,2,off-by-one
20,유재학 yujaehak16,22.08.07 18:16,10보다작을까?-1,WA,2,#include <cstdio>\nint main()\n{\n int a;\n...,1,2,off-by-one
22,유제연,22.02.12 19:45,10보다작을까?-1,WA,NaN,#include<cstdio>\nint main()\n{\nint a;\nif(10...,NOT,1,misunderstanding of the problem
26,이연준,20.11.01 18:28,10보다작을까?-1,WA,1,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,1,1,semantic error
32,전현준,22.03.11 17:21,10보다작을까?-1,WA,3,#include <cstdio>\n\nint main()\n{\nint a=11;\...,0,0,Output format mismatch
34,전현준,22.03.11 17:21,10보다작을까?-1,WA,3,#include <cstdio>\n\nint main()\n{\nint a=2;\n...,0,0,Output format mismatch
48,김무진,21.06.06 16:33,10보다작을까?-1,WA,2,#include<cstdio>\nint main()\n {\n int a...,1,1,off-by-one
50,박정빈,21.05.30 17:05,10보다작을까?-1,WA,0,#include <cstdio>\nint main()\n{\n int a;\n...,"0,1",0,Input format mismatch
51,박정빈,21.05.30 17:05,10보다작을까?-1,WA,1,#include <cstdio>\nint main()\n{\n int a;\n...,"0,1",0,semantic error
53,박정빈,21.05.30 17:07,10보다작을까?-1,WA,1,#include <cstdio>\nint main()\n{\n int a;\n...,1,1,semantic error


10보다작을까?-2

In [ ]:
"10보다작을까?-2" : {
    '0': 'off-by-one',
    '1' : 'Output format mismatch',
    '2' : 'Input format mismatch',
    '3' : 'Output format mismatch',
    '4' : 'semantic error'
}

In [94]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : off-by-one
1 : Output format mismatch
2 : Input format mismatch
3 : Output format mismatch
4 : semantic error
NaN : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "10보다작을까?-2") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]].to_string(index=False)

query1_2 = question_query  + result_query + context_query

ans1_2 = question(query1_2)
print(ans1_2)

review = [
        "misunderstanding of the problem",
        "Output format mismatch",
        "off-by-one",
        "off-by-one",
        "misunderstanding of the problem",
        "Output format mismatch",
        "off-by-one",
        "off-by-one",
        "off-by-one",
        "off-by-one",
        "off-by-one",
        "Input format mismatch",
        "Input format mismatch",
        "semantic error",
        "off-by-one"
    ]


In [157]:
review = [
        "misunderstanding of the problem",
        "Output format mismatch",
        "off-by-one",
        "off-by-one",
        "misunderstanding of the problem",
        "Output format mismatch",
        "off-by-one",
        "off-by-one",
        "off-by-one",
        "off-by-one",
        "off-by-one",
        "Input format mismatch",
        "Input format mismatch",
        "semantic error",
        "off-by-one"
    ]

ce_rows = result_df[(result_df["문제 이름"] == "10보다작을까?-2") & (result_df["정답 여부"] == "WA")]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "10보다작을까?-2") & (result_df["정답 여부"] == "WA")]


,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
87,송지훈,21.08.19 17:27,10보다작을까?-2,WA,NaN,#incldue <cstdio>\n\nint main(){\nint a;\nscan...,NOT,0,misunderstanding of the problem
89,신예린 allie610,22.10.30 13:42,10보다작을까?-2,WA,3,#include <cstdio>\nint main() {\n int a;\n ...,0,3,Output format mismatch
96,신희석 Danielshin16,22.07.22 17:51,10보다작을까?-2,WA,0,#include <cstdio>\n\nint main()\n{\n int a;...,1,0,off-by-one
98,유재학 yujaehak16,22.08.07 18:19,10보다작을까?-2,WA,0,#include <cstdio>\nint main()\n{\n int a;\n...,1,0,off-by-one
101,유제연,22.02.12 20:00,10보다작을까?-2,WA,NaN,#include<cstdio>\nint main()\n{\nint a;\nscanf...,NOT,3,misunderstanding of the problem
102,유제연,22.02.12 19:58,10보다작을까?-2,WA,3,#include<cstdio>\nint main()\n{\nint a;\nscanf...,0,3,Output format mismatch
112,허지아 jia1014,22.08.07 10:18,10보다작을까?-2,WA,0,#include <cstdio>\nint main()\n{int a;\n sc...,1,0,off-by-one
122,김무영,21.03.21 10:26,10보다작을까?-2,WA,0,"#include <cstdio>\nint main()\n{\n int a, b...",1,0,off-by-one
123,김무영,21.03.21 10:25,10보다작을까?-2,WA,0,"#include <cstdio>\nint main()\n{\n int a, b...",1,0,off-by-one
124,김무영,21.03.21 10:25,10보다작을까?-2,WA,0,"#include <cstdio>\nint main()\n{\n int a, b...",1,0,off-by-one


내나이가어때서

In [ ]:
"내나이가어때서" : {
    '0' : 'Incorrect calculation',
    '1' : 'Incorrect calculation',
    '2' : 'Output format mismatch',
    '3' : 'Incorrect calculation',
    '4' : 'Input format mismatch'
}

In [96]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Incorrect calculation
1 : Incorrect calculation
2 : Output format mismatch
3 : Incorrect calculation
4 : Input format mismatch
NaN : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "내나이가어때서") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]].to_string(index=False)

query1_3 = question_query  + result_query + context_query

ans1_3 = question(query1_3)
print(ans1_3)

Based on the given rules, the error descriptions for each code are as follows:

0: Incorrect calculation
1: Incorrect calculation
2: Output format mismatch
3: Incorrect calculation
4: Input format mismatch
NaN: Misunderstanding of the problem

Therefore, the error descriptions for the given DataFrame are:

review = [
    "Output format mismatch",      # 2
    "Output format mismatch",      # 2
    "Output format mismatch",      # 2
    "Output format mismatch",      # 2
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Input format mismatch",       # 4
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Misunderstanding of the problem",  # NaN
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 1
    "Incorrect calculation",       # 1
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Misunderstanding of the problem",  # NaN
    

In [158]:
review = [
    "Output format mismatch",      # 2
    "Output format mismatch",      # 2
    "Output format mismatch",      # 2
    "Output format mismatch",      # 2
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Input format mismatch",       # 4
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Misunderstanding of the problem",  # NaN
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 1
    "Incorrect calculation",       # 1
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Misunderstanding of the problem",  # NaN
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Output format mismatch",      # 2
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Output format mismatch",      # 2
    "Output format mismatch",      # 2
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Incorrect calculation",       # 0
    "Output format mismatch",      # 2
    "Input format mismatch",       # 4
    "Input format mismatch",       # 4
    "Incorrect calculation"        # 3
]

ce_rows = result_df[(result_df["문제 이름"] == "내나이가어때서") & (result_df["정답 여부"] == "WA")]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "내나이가어때서") & (result_df["정답 여부"] == "WA")]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
148,김강범 kkb110513,22.07.30 11:15,내나이가어때서,WA,2,#include <cstdio>\n\nint main()\n{\n int a;...,0,2,Output format mismatch
149,김강범 kkb110513,22.07.30 11:15,내나이가어때서,WA,2,#include <cstdio>\n\nint main()\n{\n int a;...,0,2,Output format mismatch
150,김강범 kkb110513,22.07.30 11:16,내나이가어때서,WA,2,#include <cstdio>\n\nint main()\n{\n int a;...,0,2,Output format mismatch
151,김강범 kkb110513,22.07.30 11:17,내나이가어때서,WA,2,#include <cstdio>\n\nint main()\n{\n int a;...,0,2,Output format mismatch
154,김동우 derek0602,22.10.22 11:08,내나이가어때서,WA,0,"#include<cstdio>\nint main(){\n int a,b;\n ...",2,0,Incorrect calculation
169,송지훈,21.08.08 20:39,내나이가어때서,WA,0,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",2,0,Incorrect calculation
172,신제민,22.03.06 15:06,내나이가어때서,WA,4,"#include<cstdio>\nint main()\n{\nint a,b;\nsca...",0,4,Input format mismatch
178,유재학 yujaehak16,22.07.23 14:11,내나이가어때서,WA,0,#include<cstdio>\nint main()\n{\n int a;\n ...,2,0,Incorrect calculation
184,이진아,21.03.18 22:13,내나이가어때서,WA,0,"#include <cstdio>\n\nint main()\n{\nint a,b;\n...",2,0,Incorrect calculation
185,이진아,21.03.18 22:01,내나이가어때서,WA,0,"#include <cstdio>\n\nint main()\n{\nint a,b;\n...",2,0,Incorrect calculation


두수비교하기(<=)

In [ ]:
"두수비교하기(<=)" : {
    '0' : 'Output format mismatch',
    '1' : 'Output format mismatch',
    '2' : 'semantic error',
    '3' : 'Incorrect calculation'
}

In [100]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Output format mismatch
2 : semantic error
3 : Incorrect calculation
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "두수비교하기(<=)") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]].to_string(index=False)

query1_4 = question_query  + result_query + context_query

ans1_4 = question(query1_4)
print(ans1_4)

review = [
    "Output format mismatch", 
    "Semantic error",
    "Incorrect calculation",
    "Output format mismatch",
    "Semantic error",
    "Semantic error",
    "Semantic error",
    "Semantic error",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Semantic error",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Incorrect calculation",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem"
]


In [159]:
review = [
    "Output format mismatch", 
    "Semantic error",
    "Incorrect calculation",
    "Output format mismatch",
    "Semantic error",
    "Semantic error",
    "Semantic error",
    "Semantic error",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Semantic error",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Incorrect calculation",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem"
]

ce_rows = result_df[(result_df["문제 이름"] == "두수비교하기(<=)") & (result_df["정답 여부"] == "WA")]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "두수비교하기(<=)") & (result_df["정답 여부"] == "WA")]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
243,김동우 derek0602,22.11.05 10:47,두수비교하기(<=),WA,1,"#include<cstdio>\nint main()\n{\n int a,b;\...",0,1,Output format mismatch
247,김민하,22.01.05 16:42,두수비교하기(<=),WA,2,"#include<cstdio>\nint main()\n{\nint a, b;\nsc...",1,2,Semantic error
249,김정찬,21.10.23 21:40,두수비교하기(<=),WA,3,"#include<cstdio>\n\nint main()\n{\nint a,b;\ns...",1,3,Incorrect calculation
251,박유빈 yoobinsik,22.11.04 19:03,두수비교하기(<=),WA,1,"#include <cstdio>\n\nint main()\n{\n int a,...",0,1,Output format mismatch
258,송지훈,21.08.29 20:48,두수비교하기(<=),WA,2,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",1,2,Semantic error
259,송지훈,21.08.29 20:48,두수비교하기(<=),WA,2,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",1,2,Semantic error
260,송지훈,21.08.29 20:47,두수비교하기(<=),WA,2,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",1,2,Semantic error
261,송지훈,21.08.29 20:46,두수비교하기(<=),WA,2,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",1,2,Semantic error
267,신준우,21.11.05 19:53,두수비교하기(<=),WA,1,"#include <stdio.h>\n\nint main()\n{\nint a,b;\...",0,1,Output format mismatch
268,신희석 Danielshin16,22.07.22 18:31,두수비교하기(<=),WA,1,"#include <cstdio>\n\nint main()\n{\n int a,...",0,1,Output format mismatch


두수비교하기(==)

In [ ]:
"두수비교하기(==)" : {
    '0' : 'Output format mismatch',
    '1' : 'Output format mismatch',
    '2': 'semantic error',
    '3' : 'Incorrect calculation',
    '4' : 'Input format mismatch'
}

In [102]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Output format mismatch
2 : semantic error
3 : Incorrect calculation
4 : Input format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "두수비교하기(==)") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]].to_string(index=False)

query1_5 = question_query  + result_query + context_query

ans1_5 = question(query1_5)
print(ans1_5)

Based on the given mappings, the error descriptions for the provided error classifications are as follows:

review = [
    "misunderstanding of the problem",
    "semantic error",
    "semantic error",
    "semantic error",
    "Input format mismatch",
    "misunderstanding of the problem",
    "semantic error",
    "Output format mismatch",
    "misunderstanding of the problem",
    "semantic error",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "Input format mismatch",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "misunderstanding of the problem",
    "semantic error",
    "Incorrect calculation",
    "Incorrect calculation",
    "semantic error",
    "Incorrect calculation",
    "Incorrect calculation",
    "semantic error",
    "misunderstanding of the problem",
    "Output format mismatch",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "Input format mismatch",
    

In [160]:
review = [
    "misunderstanding of the problem",
    "semantic error",
    "semantic error",
    "semantic error",
    "Input format mismatch",
    "misunderstanding of the problem",
    "semantic error",
    "Output format mismatch",
    "misunderstanding of the problem",
    "semantic error",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "Input format mismatch",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "misunderstanding of the problem",
    "semantic error",
    "Incorrect calculation",
    "Incorrect calculation",
    "semantic error",
    "Incorrect calculation",
    "Incorrect calculation",
    "semantic error",
    "misunderstanding of the problem",
    "Output format mismatch",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "Input format mismatch",
    "misunderstanding of the problem"
]

ce_rows = result_df[(result_df["문제 이름"] == "두수비교하기(==)") & (result_df["정답 여부"] == "WA")]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "두수비교하기(==)") & (result_df["정답 여부"] == "WA")]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
326,김강범 kkb110513,22.07.30 18:19,두수비교하기(==),WA,NaN,"#include <cstdio>\n\nint main()\n{\n int a,...",NOT,1,misunderstanding of the problem
341,박종원,21.01.10 14:39,두수비교하기(==),WA,2,#include <cstdio>\n\nint main()\n{\nint a;\nin...,1,2,semantic error
342,박종원,21.01.10 14:43,두수비교하기(==),WA,2,#include <cstdio>\n\nint main()\n{\nchar a;\nc...,1,2,semantic error
343,박종원,21.01.10 14:44,두수비교하기(==),WA,2,#include <cstdio>\n\nint main()\n{\nchar a;\nc...,1,2,semantic error
344,박종원,21.01.10 14:47,두수비교하기(==),WA,4,#include <cstdio>\n\nint main()\n{\nchar a;\nc...,0,4,Input format mismatch
345,박종원,21.01.10 14:48,두수비교하기(==),WA,NaN,#include <cstdio>\n\nint main()\n{\nint a;\nin...,NOT,2,misunderstanding of the problem
347,박종원,21.01.10 14:42,두수비교하기(==),WA,2,#include <cstdio>\n\nint main()\n{\nint a;\nin...,1,2,semantic error
349,송지훈,21.08.22 20:36,두수비교하기(==),WA,1,"#include <cstdio>\n\nint main()\n{\n\nint a, b...",1,1,Output format mismatch
353,신제민,22.03.13 14:50,두수비교하기(==),WA,NaN,"#include <cstdio>\nint main()\n{\nint a,b;\nsc...",NOT,4,misunderstanding of the problem
359,유제연,22.02.12 20:34,두수비교하기(==),WA,2,"#include<cstdio>\nint main()\n{\nint a,b;\nsca...",1,4,semantic error


두정수바꾸어출력하기

In [ ]:
"두정수바꾸어출력하기" : {
    '0' : 'misunderstanding of the problem',
    '1' : 'Incorrect calculation',
    '2' : 'Output format mismatch',
    '3' : 'Input format mismatch',
    '4' : 'Output format mismatch'
}

In [104]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : misunderstanding of the problem
1 : Incorrect calculation
2 : Output format mismatch
3 : Input format mismatch
4 : Output format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "두정수바꾸어출력하기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][:30].to_string(index=False)

query1_6 = question_query  + result_query + context_query

ans1_6 = question(query1_6)
print(ans1_6)

review = [
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "Input format mismatch",
    "Output format mismatch",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "Input format mismatch",
    "misunderstanding of the problem",
    "Input format mismatch",
    "misunderstanding of the problem",
    "Input format mismatch",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstand

In [161]:
review = [
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "Input format mismatch",
    "Output format mismatch",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "Input format mismatch",
    "misunderstanding of the problem",
    "Input format mismatch",
    "misunderstanding of the problem",
    "Input format mismatch",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch"
]

ce_rows = result_df[(result_df["문제 이름"] == "두정수바꾸어출력하기") & (result_df["정답 여부"] == "WA")][:30]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "두정수바꾸어출력하기") & (result_df["정답 여부"] == "WA")][:30]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
415,강지민 jiminkang1404,22.10.22 12:06,두정수바꾸어출력하기,WA,NaN,"#include <cstdio>\nint main()\n{\n int n,m;...",NOT,0,misunderstanding of the problem
418,권현욱,21.12.19 11:39,두정수바꾸어출력하기,WA,0,"#include <cstdio>\n\nint main()\n{\nint a,b;\n...",0,0,misunderstanding of the problem
419,김강범 kkb110513,22.07.23 18:26,두정수바꾸어출력하기,WA,0,#include <cstdio>\n\nint main()\n{\n int a;...,0,0,misunderstanding of the problem
421,김동우 derek0602,22.10.15 11:43,두정수바꾸어출력하기,WA,0,"#include<cstdio>\nint main()\n{\n int a,b;\...",0,0,misunderstanding of the problem
424,김민하,21.10.24 10:09,두정수바꾸어출력하기,WA,0,"#include <cstdio>\n\nint main()\n{\nint b,a;\n...",0,0,misunderstanding of the problem
427,김정찬,21.10.17 16:50,두정수바꾸어출력하기,WA,0,"#include<cstdio>\n\nint main()\n{\nint a,b;\ns...",0,0,misunderstanding of the problem
428,박유빈 yoobinsik,22.10.22 13:56,두정수바꾸어출력하기,WA,0,"#include <cstdio>\nint main()\n{\n int a,b;...",0,0,misunderstanding of the problem
429,박유빈 yoobinsik,22.10.22 13:56,두정수바꾸어출력하기,WA,0,"#include <cstdio>\nint main()\n{\n int a, b...",0,0,misunderstanding of the problem
432,박재윤,20.12.27 05:12,두정수바꾸어출력하기,WA,NaN,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",NOT,0,misunderstanding of the problem
435,박종원,21.01.03 13:13,두정수바꾸어출력하기,WA,0,#include<cstdio>\nint main(){\nint a;\nint b;\...,0,0,misunderstanding of the problem


In [108]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : misunderstanding of the problem
1 : Incorrect calculation
2 : Output format mismatch
3 : Input format mismatch
4 : Output format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "두정수바꾸어출력하기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][30:60].to_string(index=False)

query1_7 = question_query  + result_query + context_query

ans1_7 = question(query1_7)
print(ans1_7)

Based on the given DataFrame, the error classifications can be mapped to the error descriptions as follows:

review = [
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Input format mismatch",
    "Input format mismatch",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Incorrect calculation",
    "Output format mismatch",
    "Misunderstandi

In [162]:
review = [
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Input format mismatch",
    "Input format mismatch",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Incorrect calculation",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem"
]

ce_rows = result_df[(result_df["문제 이름"] == "두정수바꾸어출력하기") & (result_df["정답 여부"] == "WA")][30:60]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "두정수바꾸어출력하기") & (result_df["정답 여부"] == "WA")][30:60]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
466,유제연,22.01.08 19:55,두정수바꾸어출력하기,WA,2,#include<cstdio>\nint main()\n{\nint a;\nscanf...,0,2,Output format mismatch
467,유제연,22.01.08 19:58,두정수바꾸어출력하기,WA,2,#include<cstdio>\nint main()\n{\nint a;\nscanf...,0,2,Output format mismatch
468,유제연,22.01.08 19:58,두정수바꾸어출력하기,WA,2,#include<cstdio>\nint main()\n{\nint a;\nscanf...,0,2,Output format mismatch
469,유제연,22.01.08 20:16,두정수바꾸어출력하기,WA,2,"#include<cstdio>\nint main()\n{\nint b,a;\nsca...",0,1,Output format mismatch
472,이석우 jamesrhee1234567890,22.01.09 11:52,두정수바꾸어출력하기,WA,0,"#include <cstdio>\n\nint main()\n{\n int b,...",0,0,Misunderstanding of the problem
473,이석우 jamesrhee1234567890,22.01.09 11:53,두정수바꾸어출력하기,WA,0,"#include <cstdio>\n\nint main()\n{\n int b,...",0,0,Misunderstanding of the problem
475,이연준,20.10.25 17:38,두정수바꾸어출력하기,WA,0,"#include <cstdio>\n\nint main()\n{\nint a=1, b...",0,0,Misunderstanding of the problem
476,이연준,20.10.25 17:38,두정수바꾸어출력하기,WA,0,"#include <cstdio>\n\nint main()\n{\nint a=1, b...",0,0,Misunderstanding of the problem
477,이연준,20.10.25 17:39,두정수바꾸어출력하기,WA,NaN,"#include <cstdio>\n\nint main()\n{\nint a=1, b...",NOT,3,Misunderstanding of the problem
478,이연준,20.10.25 17:39,두정수바꾸어출력하기,WA,NaN,"#include <cstdio>\n\nint main()\n{\nint a=1, b...",NOT,0,Misunderstanding of the problem


In [110]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : misunderstanding of the problem
1 : Incorrect calculation
2 : Output format mismatch
3 : Input format mismatch
4 : Output format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "두정수바꾸어출력하기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][60:].to_string(index=False)

query1_8 = question_query  + result_query + context_query

ans1_8 = question(query1_8)
print(ans1_8)

review = [
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "input format mismatch", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "output format mismatch", 
    "input format mismatch", 
    "output format mismatch", 
    "input format mismatch", 
    "output format mismatch", 
    "input format mismatch", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem"
]


In [163]:
review = [
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "input format mismatch", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "output format mismatch", 
    "input format mismatch", 
    "output format mismatch", 
    "input format mismatch", 
    "output format mismatch", 
    "input format mismatch", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem", 
    "misunderstanding of the problem"
]

ce_rows = result_df[(result_df["문제 이름"] == "두정수바꾸어출력하기") & (result_df["정답 여부"] == "WA")][60:]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "두정수바꾸어출력하기") & (result_df["정답 여부"] == "WA")][60:]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
510,홍지형 aaron_moon,22.03.06 20:11,두정수바꾸어출력하기,WA,0,"#include <cstdio>\nint main()\n{\n int a,b;...",0,0,misunderstanding of the problem
511,홍지형 aaron_moon,22.03.06 20:13,두정수바꾸어출력하기,WA,0,"#include <cstdio>\nint main()\n{\n int a,b;...",0,0,misunderstanding of the problem
512,홍지형 aaron_moon,22.03.06 20:13,두정수바꾸어출력하기,WA,0,"#include <cstdio>\nint main()\n{\n int a,b;...",0,0,misunderstanding of the problem
516,강수인,21.03.07 10:25,두정수바꾸어출력하기,WA,0,"#include<cstdio>\nint main()\n{\n int a,b;\...",0,0,misunderstanding of the problem
517,김가영,21.05.23 16:34,두정수바꾸어출력하기,WA,3,"#include <cstdio>\n\nint main()\n{\n int a,...",0,3,input format mismatch
520,김무영,21.03.07 11:04,두정수바꾸어출력하기,WA,0,"#include <cstdio>\n\nint main()\n{\n int b,...",0,0,misunderstanding of the problem
521,김무영,21.03.07 11:05,두정수바꾸어출력하기,WA,0,"#include <cstdio>\n\nint main()\n{\n int b,...",0,0,misunderstanding of the problem
524,김무진,21.05.23 17:27,두정수바꾸어출력하기,WA,2,#include<cstdio>\nint main()\n{\n int a;\n ...,"0,0",2,output format mismatch
525,김무진,21.05.23 17:27,두정수바꾸어출력하기,WA,3,#include<cstdio>\nint main()\n{\n int a;\n ...,"0,0",2,input format mismatch
526,김무진,21.05.23 17:09,두정수바꾸어출력하기,WA,2,#include<cstdio>\nint main()\n{\n int a;\n ...,"0,0",2,output format mismatch


두정수입력받기

In [ ]:
"두정수입력받기" : {
    '0' : 'Output format mismatch',
    '1' : 'Output format mismatch',
    '2' : 'Output format mismatch',
    '3' : 'semantic error',
    '4' : 'Input format mismatch',
    '5' : 'Input format mismatch',
    '6' : 'Incorrect calculation',
    '7' : 'Input format mismatch'
}

In [112]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Output format mismatch
2 : Output format mismatch
3 : semantic error
4 : Input format mismatch
5 : Input format mismatch
6 : Incorrect calculation
7 : Input format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem


"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "두정수입력받기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][:20].to_string(index=False)

query1_9 = question_query  + result_query + context_query

ans1_9 = question(query1_9)
print(ans1_9)

review = [
        "misunderstanding of the problem",
        "Output format mismatch",
        "Input format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Input format mismatch",
        "Input format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Input format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "misunderstanding of the problem"
       ]


In [164]:
review = [
        "misunderstanding of the problem",
        "Output format mismatch",
        "Input format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Input format mismatch",
        "Input format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Input format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "misunderstanding of the problem"
       ]

ce_rows = result_df[(result_df["문제 이름"] == "두정수입력받기") & (result_df["정답 여부"] == "WA")][:20]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "두정수입력받기") & (result_df["정답 여부"] == "WA")][:20]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
551,강지민 jiminkang1404,22.10.22 11:41,두정수입력받기,WA,NaN,"#include <cstdio>\nint main()\n{\n int n,m;...",NOT,2,misunderstanding of the problem
555,권현욱,21.12.19 11:36,두정수입력받기,WA,0,"#include <cstdio>\n\nint main()\n{\nint a,b;\n...",0,0,Output format mismatch
558,김민하,21.10.17 13:23,두정수입력받기,WA,5,#include<cstdio>\nint main()\n{\nint n;\nscanf...,0,5,Input format mismatch
561,김승희,20.11.12 22:06,두정수입력받기,WA,1,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",0,1,Output format mismatch
562,김승희,20.11.12 22:05,두정수입력받기,WA,1,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",0,1,Output format mismatch
564,김정찬,21.10.17 16:30,두정수입력받기,WA,0,#include <cstdio>\n\nint main()\n{\nint a;\nin...,0,7,Output format mismatch
574,박윤선,21.07.24 20:56,두정수입력받기,WA,0,"#include <cstdio>\n\nint main()\n{\nint a,b;\n...",0,0,Output format mismatch
576,박재윤,20.12.23 22:36,두정수입력받기,WA,2,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",0,3,Output format mismatch
578,박종원,20.12.27 15:32,두정수입력받기,WA,4,#include <cstdio>\n\nint main()\n{\nint a=-7;\...,0,4,Input format mismatch
579,박종원,21.01.01 14:17,두정수입력받기,WA,4,#include <cstdio>\n\nint main()\n\n{\nint a=-7...,0,4,Input format mismatch


In [114]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Output format mismatch
2 : Output format mismatch
3 : semantic error
4 : Input format mismatch
5 : Input format mismatch
6 : Incorrect calculation
7 : Input format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem


"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "두정수입력받기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][20:40].to_string(index=False)

query1_10 = question_query  + result_query + context_query

ans1_10 = question(query1_10)
print(ans1_10)

review = [
        "Input format mismatch",
        "Input format mismatch",
        "Semantic error",
        "Input format mismatch",
        "Output format mismatch",
        "Semantic error",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Input format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Semantic error",
        "Output format mismatch",
        "Semantic error",
        "Semantic error",
        "Output format mismatch"
        ]


In [165]:
review = [
        "Input format mismatch",
        "Input format mismatch",
        "Semantic error",
        "Input format mismatch",
        "Output format mismatch",
        "Semantic error",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Input format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Semantic error",
        "Output format mismatch",
        "Semantic error",
        "Semantic error",
        "Output format mismatch"
        ]
ce_rows = result_df[(result_df["문제 이름"] == "두정수입력받기") & (result_df["정답 여부"] == "WA")][20:40]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "두정수입력받기") & (result_df["정답 여부"] == "WA")][20:40]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
608,이연준,20.10.25 17:29,두정수입력받기,WA,5,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,0,5,Input format mismatch
609,이연준,20.10.25 17:29,두정수입력받기,WA,5,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,0,5,Input format mismatch
611,임선균,22.01.06 20:49,두정수입력받기,WA,3,#include<cstdio>\nint main()\n{\nint n;\nscanf...,0,5,Semantic error
614,전현준,22.03.06 19:40,두정수입력받기,WA,5,#include<cstdio>\nint main()\n{\nint n;\nscanf...,0,5,Input format mismatch
617,정진욱 HARRY,22.03.13 19:47,두정수입력받기,WA,2,"#include <cstdio>\n\nint main()\n{\n int a,...",0,0,Output format mismatch
618,정진욱 HARRY,22.03.13 19:48,두정수입력받기,WA,3,"#include <cstdio>\n\nint main()\n{\n int a,...",0,6,Semantic error
619,정진욱 HARRY,22.03.13 19:51,두정수입력받기,WA,NaN,"#include <cstdio>\n\nint main()\n{\n int a,...",NOT,2,Misunderstanding of the problem
620,정진욱 HARRY,22.03.13 19:52,두정수입력받기,WA,NaN,"#include <cstdio>\n\nint main()\n{\n int a,...",NOT,2,Misunderstanding of the problem
621,정진욱 HARRY,22.03.26 12:25,두정수입력받기,WA,NaN,"#include <cstdio>\n\nint main()\n{\n int a,...",NOT,2,Misunderstanding of the problem
622,정진욱 HARRY,22.03.26 12:34,두정수입력받기,WA,NaN,"#include <cstdio>\n\nint main()\n{\n int a,...",NOT,2,Misunderstanding of the problem


In [116]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Output format mismatch
2 : Output format mismatch
3 : semantic error
4 : Input format mismatch
5 : Input format mismatch
6 : Incorrect calculation
7 : Input format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem


"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "두정수입력받기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][40:].to_string(index=False)

query1_11 = question_query  + result_query + context_query

ans1_11 = question(query1_11)
print(ans1_11)

Based on the given mapping, the error descriptions for the provided dataframe are as follows:

review = [
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "Input format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "semantic error",
        "Output format mismatch",
        "misunderstanding of the problem",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch

In [166]:
review = [
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "Input format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "semantic error",
        "semantic error",
        "Output format mismatch",
        "misunderstanding of the problem",
        "Output format mismatch",
        "semantic error",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "semantic error"
]
ce_rows = result_df[(result_df["문제 이름"] == "두정수입력받기") & (result_df["정답 여부"] == "WA")][40:]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "두정수입력받기") & (result_df["정답 여부"] == "WA")][40:]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
646,김가영,21.05.23 16:27,두정수입력받기,WA,3,"#include <cstdio>\n\nint main()\n{\n int a,...","0,0",0,semantic error
647,김가영,21.05.23 16:26,두정수입력받기,WA,0,"#include <cstdio>\n\nint main()\n{\n int a,...","0,0",0,Output format mismatch
648,김가영,21.05.23 16:26,두정수입력받기,WA,3,"#include <cstdio>\n\nint main()\n{\n int a,...","0,0",0,semantic error
649,김가영,21.05.23 16:23,두정수입력받기,WA,0,"#include <cstdio>\n\nint main()\n{\n int a,...","0,0",0,Output format mismatch
650,김가영,21.05.23 16:23,두정수입력받기,WA,3,"#include <cstdio>\n\nint main()\n{\n int a,...","0,0",0,semantic error
652,김근태,21.05.30 15:08,두정수입력받기,WA,4,#include <cstdio>\n\nint main()\n{\n printf...,"0,0",6,Input format mismatch
653,김근태,21.05.30 15:08,두정수입력받기,WA,6,#include <cstdio>\n\nint main()\n{\n printf...,"0,0",6,Incorrect calculation
655,김무영,21.03.07 10:56,두정수입력받기,WA,0,"#include <cstdio>\n\nint main()\n{\n int a,...","0,0",2,Output format mismatch
656,김무영,21.03.07 10:56,두정수입력받기,WA,3,"#include <cstdio>\n\nint main()\n{\n int a,...","0,0",2,semantic error
657,김무영,21.03.07 10:53,두정수입력받기,WA,0,"#include <cstdio>\n\nint main()\n{\n int a,...","0,0",2,Output format mismatch


반값이벤트

In [ ]:
"반값이벤트" : {
    '0' : 'Output format mismatch',
    '1' : 'Incorrect calculation',
    '2' : 'Output format mismatch',
    '3' : 'Input format mismatch'

}

In [118]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Incorrect calculation
2 : Output format mismatch
3 : Input format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "반값이벤트") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]].to_string(index=False)

query1_12 = question_query  + result_query + context_query

ans1_12 = question(query1_12)
print(ans1_12)

Based on the provided information and rules, the error descriptions for the given error codes are as follows:

review = [
        "misunderstanding of the problem",
        "Incorrect calculation",
        "Incorrect calculation",
        "misunderstanding of the problem",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Input format mismatch",
        "Incorrect calculation",
        "Input format mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "misunderstanding of the problem",
        "misunderstanding of the problem",
        "Incorrect calculation",
        "Output format mismatch",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch"
        ]


In [167]:
review = [
        "misunderstanding of the problem",
        "Incorrect calculation",
        "Incorrect calculation",
        "misunderstanding of the problem",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Input format mismatch",
        "Incorrect calculation",
        "Input format mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "misunderstanding of the problem",
        "misunderstanding of the problem",
        "Incorrect calculation",
        "Output format mismatch",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch"
        ]

ce_rows = result_df[(result_df["문제 이름"] == "반값이벤트") & (result_df["정답 여부"] == "WA")]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "반값이벤트") & (result_df["정답 여부"] == "WA")]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
1218,강지민 jiminkang1404,22.11.12 16:20,반값이벤트,WA,NaN,"#include <cstdio>\nint main()\n{\n int a,b;...",NOT,1,misunderstanding of the problem
1220,김강범 kkb110513,22.07.30 10:58,반값이벤트,WA,1,#include <cstdio>\n\nint main()\n{\n int a;...,2,3,Incorrect calculation
1237,신승명 himond1126,22.07.16 17:24,반값이벤트,WA,1,#include<cstdio>\n\nint main()\n{\n int a=5...,2,1,Incorrect calculation
1245,유재학 yujaehak16,22.07.23 13:18,반값이벤트,WA,NaN,#include <cstdio>\nint main()\n{\n int a;\n...,NOT,2,misunderstanding of the problem
1246,유재학 yujaehak16,22.07.23 13:21,반값이벤트,WA,0,#include <cstdio>\nint main()\n{\n int a;\n...,"0,2",2,Output format mismatch
1247,유재학 yujaehak16,22.07.23 13:21,반값이벤트,WA,1,#include <cstdio>\nint main()\n{\n int a;\n...,"0,2",2,Incorrect calculation
1248,유재학 yujaehak16,22.07.23 13:23,반값이벤트,WA,0,#include <cstdio>\nint main()\n{\n int a;\n...,"0,2",2,Output format mismatch
1249,유재학 yujaehak16,22.07.23 13:23,반값이벤트,WA,1,#include <cstdio>\nint main()\n{\n int a;\n...,"0,2",2,Incorrect calculation
1250,유재학 yujaehak16,22.07.23 13:24,반값이벤트,WA,0,#include <cstdio>\nint main()\n{\n int a;\n...,0,2,Output format mismatch
1251,유재학 yujaehak16,22.07.23 13:24,반값이벤트,WA,0,#include <cstdio>\nint main()\n{\n int a;\n...,0,2,Output format mismatch


사칙연산

In [ ]:
"사칙연산" : {
    '0' : 'Output format mismatch',
    '1' : 'Output format mismatch',
    '2' : 'Output format mismatch',
    '3' : 'Incorrect calculation',
    '4' : 'Output format mismatch'
}

In [120]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Output format mismatch
2 : Output format mismatch
3 : Incorrect calculation
4 : Output format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  \
    + df.loc[(df["문제 이름"] == "사칙연산") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][:20].to_string(index=False)

query1_13 = question_query  + result_query + context_query

ans1_13 = question(query1_13)
print(ans1_13)

Based on the rules provided, the error descriptions for each code in the DataFrame are as follows:

review = [
    "Incorrect calculation",
    "Incorrect calculation",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Output format mismatch"
]


In [168]:
review = [
    "Incorrect calculation",
    "Incorrect calculation",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Output format mismatch"
]

ce_rows = result_df[(result_df["문제 이름"] == "사칙연산") & (result_df["정답 여부"] == "WA")][:20]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "사칙연산") & (result_df["정답 여부"] == "WA")][:20]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
1304,강지민 jiminkang1404,22.10.25 21:31,사칙연산,WA,3,"#include <cstdio>\nint main()\n{\n int a,b;...",0,1,Incorrect calculation
1305,강지민 jiminkang1404,22.10.29 10:07,사칙연산,WA,3,"#include <cstdio>\nint main()\n{\n int a,b;...",0,1,Incorrect calculation
1309,김강범 kkb110513,22.07.26 12:55,사칙연산,WA,NaN,#include <cstdio>\n\nint main()\n{\n int a;...,NOT,0,Misunderstanding of the problem
1316,김승희,20.11.13 15:15,사칙연산,WA,NaN,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",NOT,4,Misunderstanding of the problem
1317,김승희,20.11.13 15:15,사칙연산,WA,1,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",2,1,Output format mismatch
1318,김정찬,21.10.17 17:42,사칙연산,WA,0,"#include <cstdio>\n\nint main()\n{\nint a,b;\n...",0,0,Output format mismatch
1319,김정찬,21.10.17 17:45,사칙연산,WA,4,"#include <cstdio>\n\nint main()\n{\nint a,b;\n...",0,2,Output format mismatch
1320,김정찬,21.10.17 17:46,사칙연산,WA,4,"#include <cstdio>\n\nint main()\n{\nint a,b;\n...",0,2,Output format mismatch
1327,박윤선,21.07.15 10:06,사칙연산,WA,1,"#include<stdio.h>\nint main() {\nint a,b;\nsca...",2,1,Output format mismatch
1329,박재윤,21.01.05 23:50,사칙연산,WA,NaN,"#include <cstdio>\n\nint main()\n{\nint a, b; ...",NOT,0,Misunderstanding of the problem


In [122]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Output format mismatch
2 : Output format mismatch
3 : Incorrect calculation
4 : Output format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "사칙연산") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][20:40].to_string(index=False)

query1_14 = question_query  + result_query + context_query

ans1_14 = question(query1_14)
print(ans1_14)

Based on the given Dataframe, the error classifications can be mapped to the error descriptions as follows:

review = [
        "misunderstanding of the problem",    # NaN
        "misunderstanding of the problem",    # NaN
        "Incorrect calculation",              # 3
        "Output format mismatch",             # 2
        "Output format mismatch",             # 2
        "Output format mismatch",             # 2
        "Output format mismatch",             # 2
        "Output format mismatch",             # 0
        "Output format mismatch",             # 0
        "Output format mismatch",             # 0
        "Output format mismatch",             # 2
        "Output format mismatch",             # 0
        "Output format mismatch",             # 2
        "Output format mismatch",             # 2
        "Output format mismatch",             # 2
        "Output format mismatch",             # 2
        "Output format mismatch",             # 0
        "Output format mis

In [169]:
review = [
        "misunderstanding of the problem",    # NaN
        "misunderstanding of the problem",    # NaN
        "Incorrect calculation",              # 3
        "Output format mismatch",             # 2
        "Output format mismatch",             # 2
        "Output format mismatch",             # 2
        "Output format mismatch",             # 2
        "Output format mismatch",             # 0
        "Output format mismatch",             # 0
        "Output format mismatch",             # 0
        "Output format mismatch",             # 2
        "Output format mismatch",             # 0
        "Output format mismatch",             # 2
        "Output format mismatch",             # 2
        "Output format mismatch",             # 2
        "Output format mismatch",             # 2
        "Output format mismatch",             # 0
        "Output format mismatch",             # 2
        "misunderstanding of the problem",    # NaN
        "Output format mismatch"              # 0
]

ce_rows = result_df[(result_df["문제 이름"] == "사칙연산") & (result_df["정답 여부"] == "WA")][20:40]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "사칙연산") & (result_df["정답 여부"] == "WA")][20:40]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
1356,임선균,22.01.06 21:50,사칙연산,WA,NaN,"#include<cstdio>\nint main()\n{\nint 3,2;\nsca...",NOT,3,misunderstanding of the problem
1357,임선균,22.01.06 21:57,사칙연산,WA,NaN,"#include<cstdio>\nint main()\n{\nint 3,5;\nsca...",NOT,2,misunderstanding of the problem
1358,임선균,22.01.09 13:23,사칙연산,WA,3,"#include<cstdio>\nint main()\n{\nint 3,2;\nsca...",0,2,Incorrect calculation
1359,임선균,22.01.09 13:32,사칙연산,WA,2,"#include<cstdio>\nint main()\n{\nint a,b,;\nsc...",0,2,Output format mismatch
1362,임선균,22.01.09 13:41,사칙연산,WA,2,"#include<cstdio>\n\nint main()\n{\nint a,b;\ns...",0,2,Output format mismatch
1363,임선균,22.01.09 13:43,사칙연산,WA,2,"#include<cstdio>\n\nint main()\n{\nint a,b;\ns...",0,2,Output format mismatch
1364,임선균,22.01.09 13:42,사칙연산,WA,2,"#include<cstdio>\n\nint main()\n{\nint a,b;\ns...",0,2,Output format mismatch
1366,전현준,22.03.06 20:50,사칙연산,WA,0,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",0,0,Output format mismatch
1367,전현준,22.03.06 20:50,사칙연산,WA,0,"#include <cstdio>\n\nint main()\n{\nint a, b;\...",0,0,Output format mismatch
1369,정진욱 HARRY,22.07.28 20:06,사칙연산,WA,0,"#include <cstdio>\n\nint main()\n{\n int a,...","0,0",0,Output format mismatch


In [123]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Output format mismatch
2 : Output format mismatch
3 : Incorrect calculation
4 : Output format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "사칙연산") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][40:60].to_string(index=False)

query1_15 = question_query  + result_query + context_query

ans1_15 = question(query1_15)
print(ans1_15)

Based on the given rules, the error descriptions for each error classification in the DataFrame are as follows:

review = [
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Incorrect calculation",
    "Output format mismatch",
    "Incorrect calculation",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch"
]


In [170]:
review = [
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Incorrect calculation",
    "Output format mismatch",
    "Incorrect calculation",
    "Misunderstanding of the problem",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Misunderstanding of the problem",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch",
    "Output format mismatch"
]

ce_rows = result_df[(result_df["문제 이름"] == "사칙연산") & (result_df["정답 여부"] == "WA")][40:60]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "사칙연산") & (result_df["정답 여부"] == "WA")][40:60]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
1387,홍지형 aaron_moon,22.03.13 18:14,사칙연산,WA,2,"#include <cstdio>\n\nint main()\n{\n int a,...","0,0",0,Output format mismatch
1388,홍지형 aaron_moon,22.03.13 18:17,사칙연산,WA,NaN,"#include <cstdio>\n\nint main()\n{\n int a,...",NOT,3,Misunderstanding of the problem
1391,강석준,21.05.23 17:15,사칙연산,WA,0,"#include <cstdio>\n\nint main()\n{\n int a,...","0,0",0,Output format mismatch
1392,강석준,21.05.23 17:15,사칙연산,WA,2,"#include <cstdio>\n\nint main()\n{\n int a,...","0,0",0,Output format mismatch
1393,강석준,21.05.23 17:32,사칙연산,WA,0,"#include <cstdio>\n\nint main()\n{\n int a,...",0,0,Misunderstanding of the problem
1394,강석준,21.07.18 16:45,사칙연산,WA,-1,"#include<cstdio>\n\nint main()\n{\n int a,b...",NOT,2,Misunderstanding of the problem
1395,강석준,21.05.23 17:34,사칙연산,WA,3,"#include <cstdio>\n\nint main()\n{\n int a,...",0,3,Incorrect calculation
1396,강석준,21.05.25 22:19,사칙연산,WA,4,"#include<cstdio>\n\nint main()\n{\n int a,b...",0,2,Output format mismatch
1397,강석준,21.05.23 17:38,사칙연산,WA,3,"#include <cstdio>\n\nint main()\n{\n int a,...",0,3,Incorrect calculation
1398,강수인,21.02.18 19:54,사칙연산,WA,-1,#include<cstdio>\nint main()\n{\n int a;\n ...,NOT,2,Misunderstanding of the problem


In [126]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Output format mismatch
2 : Output format mismatch
3 : Incorrect calculation
4 : Output format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "사칙연산") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][60:].to_string(index=False)

query1_16 = question_query  + result_query + context_query

ans1_16 = question(query1_16)
print(ans1_16)

review = [
    "Output format mismatch", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Incorrect calculation", 
    "Output format mismatch", 
    "Incorrect calculation", 
    "Incorrect calculation", 
    "Incorrect calculation", 
    "Output format mismatch", 
    "Incorrect calculation", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Misunderstanding of the problem", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Misunderstanding of the problem", 
    "Misunderstanding of the problem", 
    "Misunderstanding of the problem", 
    "Incorrect calculation", 
    "Incorrect calculation", 
    "Misunderstanding of the problem", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Misunderstanding of the problem", 
    "Misunderstanding of the problem", 
   

In [171]:
review = [
    "Output format mismatch", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Incorrect calculation", 
    "Output format mismatch", 
    "Incorrect calculation", 
    "Incorrect calculation", 
    "Incorrect calculation", 
    "Output format mismatch", 
    "Incorrect calculation", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Misunderstanding of the problem", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Misunderstanding of the problem", 
    "Misunderstanding of the problem", 
    "Misunderstanding of the problem", 
    "Incorrect calculation", 
    "Incorrect calculation", 
    "Misunderstanding of the problem", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Output format mismatch", 
    "Misunderstanding of the problem", 
    "Misunderstanding of the problem", 
    "Output format mismatch", 
    "Output format mismatch"
]

ce_rows = result_df[(result_df["문제 이름"] == "사칙연산") & (result_df["정답 여부"] == "WA")][60:]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "사칙연산") & (result_df["정답 여부"] == "WA")][60:]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
1416,김무진,21.05.23 18:30,사칙연산,WA,0,"#include<cstdio>\n\nint main()\n{\n int a, ...","0,0",0,Output format mismatch
1417,김무진,21.05.23 18:30,사칙연산,WA,2,"#include<cstdio>\n\nint main()\n{\n int a, ...","0,0",0,Output format mismatch
1418,김무진,21.05.23 18:27,사칙연산,WA,0,"#include<cstdio>\n\nint main()\n{\n int a, ...","0,0",0,Output format mismatch
1419,김무진,21.05.23 18:27,사칙연산,WA,2,"#include<cstdio>\n\nint main()\n{\n int a, ...","0,0",0,Incorrect calculation
1423,박민호,21.05.16 18:14,사칙연산,WA,2,"#include<cstdio>\nint main()\n{\n int a,b,c...",0,2,Output format mismatch
1424,박정빈,21.05.23 17:29,사칙연산,WA,3,"#include <cstdio>\nint main()\n{\n int a,b;...",0,3,Incorrect calculation
1425,박정빈,21.05.23 17:30,사칙연산,WA,3,"#include <cstdio>\nint main()\n{\n int a,b;...",0,3,Incorrect calculation
1426,박정빈,21.05.23 17:41,사칙연산,WA,3,"#include <cstdio>\nint main()\n{\n int a,b;...",0,3,Incorrect calculation
1431,박준,21.03.07 11:34,사칙연산,WA,2,#include <cstdio>\n\nint main()\n{\n int a;...,"0,0",3,Output format mismatch
1432,박준,21.03.07 11:34,사칙연산,WA,3,#include <cstdio>\n\nint main()\n{\n int a;...,"0,0",3,Incorrect calculation


시간단위로변환하기

In [ ]:
"시간단위로변환하기" : {
    '0' : 'Incorrect calculation',
    '1' : 'Incorrect calculation',
    '2' : 'Input format mismatch',
    '3' : 'Output format mismatch'
}

In [128]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Incorrect calculation
1 : Incorrect calculation
2 : Input format mismatch
3 : Output format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "시간단위로변환하기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]].to_string(index=False)

query1_17 = question_query  + result_query + context_query

ans1_17 = question(query1_17)
print(ans1_17)

Based on the error classifications provided, the corresponding error descriptions for the given DataFrame are as follows:

review = [
    "Incorrect calculation",
    "Misunderstanding of the problem",
    "Input format mismatch",
    "Input format mismatch",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "Input format mismatch",
    "Input format mismatch",
    "Input format mismatch",
    "Incorrect calculation",
    "Incorrect calculation",
    "Input format mismatch",
    "Input format mismatch",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "Input format mismatch",
    "Incorrect calculation",
    "Input format mismatch",
    "Incorrect calculation",
    "Input format mismatch",
    "Incorrect calculation",
    "Input format mismatch",
    "Incorrect calculation",
    "Incorrect 

In [172]:
review = [
    "Incorrect calculation",
    "Misunderstanding of the problem",
    "Input format mismatch",
    "Input format mismatch",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "Input format mismatch",
    "Input format mismatch",
    "Input format mismatch",
    "Incorrect calculation",
    "Incorrect calculation",
    "Input format mismatch",
    "Input format mismatch",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "Incorrect calculation",
    "Input format mismatch",
    "Incorrect calculation",
    "Input format mismatch",
    "Incorrect calculation",
    "Input format mismatch",
    "Incorrect calculation",
    "Input format mismatch",
    "Incorrect calculation",
    "Incorrect calculation"
]

ce_rows = result_df[(result_df["문제 이름"] == "시간단위로변환하기") & (result_df["정답 여부"] == "WA")]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "시간단위로변환하기") & (result_df["정답 여부"] == "WA")]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
706,김정찬,21.10.17 17:28,시간단위로변환하기,WA,1,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,2,0,Incorrect calculation
707,김정찬,21.10.17 17:30,시간단위로변환하기,WA,NaN,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,NOT,0,Misunderstanding of the problem
711,박재윤,21.01.02 23:16,시간단위로변환하기,WA,2,"#include <cstdio>\n\nint main()\n{\nint a, b; ...",0,4,Input format mismatch
713,박재윤,21.01.02 23:27,시간단위로변환하기,WA,2,"#include <cstdio>\n\nint main()\n{\nint a, b; ...",0,4,Input format mismatch
720,신제민,22.03.06 13:55,시간단위로변환하기,WA,1,#include<cstdio>\nint main()\n{\nint a;\na<100...,2,0,Incorrect calculation
721,신제민,22.03.06 13:56,시간단위로변환하기,WA,1,#include<cstdio>\nint main()\n{\nint a;\nscanf...,2,0,Incorrect calculation
722,신제민,22.03.06 13:56,시간단위로변환하기,WA,1,#include<cstdio>\nint main()\n{\nint a;\nscanf...,2,0,Incorrect calculation
723,신제민,22.03.06 13:54,시간단위로변환하기,WA,1,#include<cstdio>\nint main()\n{\nint a;\nscanf...,2,0,Incorrect calculation
724,신제민,22.03.06 13:53,시간단위로변환하기,WA,1,#include<cstdio>\nint main()\n{\nint a;\nscanf...,2,0,Incorrect calculation
725,신제민,22.03.06 13:52,시간단위로변환하기,WA,1,#include<cstdio>\nint main()\n{\nint a;\nscanf...,2,0,Incorrect calculation


원의넓이구하기

In [ ]:
"원의넓이구하기" : {
    '0' : 'type mismatch'
    '1' : 'type mismatch'
    '2' : 'type mismatch'
    '3' : 'Output format mismatch'
    '4' : 'Incorrect calculation'
}

In [130]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : type mismatch
1 : type mismatch
2 : type mismatch
3 : Output format mismatch
4 : Incorrect calculation
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "원의넓이구하기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][:20].to_string(index=False)

query1_18 = question_query  + result_query + context_query

ans1_18 = question(query1_18)
print(ans1_18)

review = [
        "type mismatch",
        "misunderstanding of the problem",
        "misunderstanding of the problem",
        "type mismatch",
        "type mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "misunderstanding of the problem",
        "type mismatch",
        "type mismatch",
        "misunderstanding of the problem",
        "type mismatch"
]


In [173]:
review = [
        "type mismatch",
        "misunderstanding of the problem",
        "misunderstanding of the problem",
        "type mismatch",
        "type mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "misunderstanding of the problem",
        "type mismatch",
        "type mismatch",
        "misunderstanding of the problem",
        "type mismatch"
]

ce_rows = result_df[(result_df["문제 이름"] == "원의넓이구하기") & (result_df["정답 여부"] == "WA")][:20]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "원의넓이구하기") & (result_df["정답 여부"] == "WA")][:20]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
785,권현욱,21.12.19 12:18,원의넓이구하기,WA,2,#include <cstdio>\n\nint main()\n{\nint a;\n\n...,0,2,type mismatch
786,권현욱,21.12.19 12:20,원의넓이구하기,WA,NaN,#include <cstdio>\n\nint main()\n{\ndouble a;\...,NOT,4,misunderstanding of the problem
787,권현욱,21.12.19 12:19,원의넓이구하기,WA,NaN,#include <cstdio>\n\nint main()\n{\ndouble a;\...,NOT,4,misunderstanding of the problem
789,권현욱,21.12.19 12:19,원의넓이구하기,WA,0,#include <cstdio>\n\nint main()\n{\nint a;\n\n...,0,0,type mismatch
790,김강범 kkb110513,22.07.30 16:43,원의넓이구하기,WA,2,#include <cstdio>\n\nint main()\n{\n double...,"0,2",2,type mismatch
791,김강범 kkb110513,22.07.30 16:43,원의넓이구하기,WA,4,#include <cstdio>\n\nint main()\n{\n double...,"0,2",2,Incorrect calculation
792,김강범 kkb110513,22.07.30 16:43,원의넓이구하기,WA,4,#include <cstdio>\n\nint main()\n{\n double...,2,3,Incorrect calculation
793,김강범 kkb110513,22.07.30 16:44,원의넓이구하기,WA,2,#include <cstdio>\n\nint main()\n{\n double...,0,2,type mismatch
794,김강범 kkb110513,22.07.30 16:44,원의넓이구하기,WA,3,#include <cstdio>\n\nint main()\n{\n double...,0,3,Output format mismatch
796,김동우 derek0602,22.11.06 22:49,원의넓이구하기,WA,0,#include<cstdio>\nint main(){\n int r;\n ...,0,0,type mismatch


In [132]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : type mismatch
1 : type mismatch
2 : type mismatch
3 : Output format mismatch
4 : Incorrect calculation
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "원의넓이구하기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][20:40].to_string(index=False)

query1_19 = question_query  + result_query + context_query

ans1_19 = question(query1_19)
print(ans1_19)

Based on the given data, the corresponding error descriptions for each error code are as follows:

0 : type mismatch
1 : type mismatch
2 : type mismatch
3 : Output format mismatch
4 : Incorrect calculation
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

So, the list of error descriptions for the given DataFrame are:

review = [
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "type mismatch",
        "misunderstanding of the problem",
        "misunderstanding of the problem",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch"
        ]


In [174]:
review = [
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "type mismatch",
        "misunderstanding of the problem",
        "misunderstanding of the problem",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch"
        ]

ce_rows = result_df[(result_df["문제 이름"] == "원의넓이구하기") & (result_df["정답 여부"] == "WA")][20:40]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "원의넓이구하기") & (result_df["정답 여부"] == "WA")][20:40]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
812,박유빈 yoobinsik,22.12.16 22:10,원의넓이구하기,WA,0,#include<cstdio>\n\nint main()\n{\n int r;\...,0,0,type mismatch
813,박유빈 yoobinsik,22.12.16 22:11,원의넓이구하기,WA,0,#include <cstdio>\n\nint main()\n{\n int r;...,0,0,type mismatch
814,박유빈 yoobinsik,22.12.17 16:08,원의넓이구하기,WA,0,#include <cstdio>\n\nint main()\n{\n int r;...,0,0,type mismatch
817,박재윤,21.01.09 19:30,원의넓이구하기,WA,3,#include<cstdio>\nint main()\n{\ndouble a;\nin...,0,3,Output format mismatch
818,박재윤,21.01.09 19:30,원의넓이구하기,WA,3,#include<cstdio>\nint main()\n{\ndouble a;\nin...,0,3,Output format mismatch
819,박재윤,21.01.09 19:22,원의넓이구하기,WA,3,#include<cstdio>\nint main()\n{\ndouble a;\nin...,0,3,Output format mismatch
820,박재윤,21.01.09 19:21,원의넓이구하기,WA,3,#include<cstdio>\nint main()\n{\ndouble a;\nin...,0,3,Output format mismatch
821,박재윤,21.01.09 19:21,원의넓이구하기,WA,3,#include<cstdio>\nint main()\n{\ndouble a;\nin...,0,3,Output format mismatch
826,송지원,20.05.27 23:48,원의넓이구하기,WA,0,#include <cstdio>\nint main(){\nint r;\nscanf(...,0,0,type mismatch
828,송지훈,21.08.09 11:56,원의넓이구하기,WA,NaN,#include <cstdio>\n\nint main()\n{\ndouble a=5...,NOT,2,misunderstanding of the problem


In [133]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : type mismatch
1 : type mismatch
2 : type mismatch
3 : Output format mismatch
4 : Incorrect calculation
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "원의넓이구하기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][40:60].to_string(index=False)

query1_20 = question_query  + result_query + context_query

ans1_20 = question(query1_20)
print(ans1_20)

review = [
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch"
]


In [175]:
review = [
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch"
]

ce_rows = result_df[(result_df["문제 이름"] == "원의넓이구하기") & (result_df["정답 여부"] == "WA")][40:60]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "원의넓이구하기") & (result_df["정답 여부"] == "WA")][40:60]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
848,이석우 jamesrhee1234567890,22.01.16 15:19,원의넓이구하기,WA,0,#include <cstdio>\nint main()\n{\n int a;\n...,0,0,type mismatch
849,이석우 jamesrhee1234567890,22.01.16 15:19,원의넓이구하기,WA,0,#include <cstdio>\nint main()\n{\n int r;\n...,0,0,type mismatch
850,이석우 jamesrhee1234567890,22.01.16 15:25,원의넓이구하기,WA,0,#include <cstdio>\nint main()\n{\n int r;\n...,0,0,type mismatch
851,이석우 jamesrhee1234567890,22.01.16 15:26,원의넓이구하기,WA,1,#include <cstdio>\nint main()\n{\n double r...,0,1,Output format mismatch
852,이석우 jamesrhee1234567890,22.01.22 17:08,원의넓이구하기,WA,0,#include <cstdio>\nint main()\n{\n int r;\n...,0,0,type mismatch
853,이석우 jamesrhee1234567890,22.01.22 17:10,원의넓이구하기,WA,0,#include <cstdio>\nint main()\n{\n int r;\n...,0,0,type mismatch
854,이석우 jamesrhee1234567890,22.01.22 17:10,원의넓이구하기,WA,0,#include <cstdio>\nint main()\n{\n int r;\n...,0,0,type mismatch
855,이석우 jamesrhee1234567890,22.01.22 17:12,원의넓이구하기,WA,1,#include <cstdio>\nint main()\n{\n double r...,0,1,Output format mismatch
860,이진아,21.03.20 15:12,원의넓이구하기,WA,2,#include <cstdio>\n\nint main()\n{\ndouble r;\...,0,2,type mismatch
861,이진아,21.03.20 15:15,원의넓이구하기,WA,3,#include <cstdio>\n\nint main()\n{\ndouble r;\...,0,3,Output format mismatch


In [137]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : type mismatch
1 : type mismatch
2 : type mismatch
3 : Output format mismatch
4 : Incorrect calculation
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "원의넓이구하기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][60:].to_string(index=False)

query1_21 = question_query  + result_query + context_query

ans1_21 = question(query1_21)
print(ans1_21)

review = [
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "misunderstanding of the problem",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch"
        ]


In [176]:
review = [
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "misunderstanding of the problem",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "Output format mismatch",
        "type mismatch",
        "type mismatch",
        "type mismatch"
        ]

ce_rows = result_df[(result_df["문제 이름"] == "원의넓이구하기") & (result_df["정답 여부"] == "WA")][60:]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "원의넓이구하기") & (result_df["정답 여부"] == "WA")][60:]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
875,정시원,22.01.09 10:27,원의넓이구하기,WA,0,#include <cstdio>\n\nint main()\n{\nint r;\nsc...,0,0,type mismatch
876,정시원,22.01.09 10:20,원의넓이구하기,WA,0,#include <cstdio>\n\nint main()\n{\nint r;\nsc...,0,0,type mismatch
877,정시원,22.01.09 10:17,원의넓이구하기,WA,0,#include <cstdio>\n\nint main()\n{\nint r;\nsc...,0,0,type mismatch
881,최도윤,21.10.15 17:27,원의넓이구하기,WA,0,#include <cstdio>\n\nint main() {\nint r;\nsca...,0,0,type mismatch
882,최도윤,21.10.15 17:25,원의넓이구하기,WA,0,#include <cstdio>\n\nint main() {\nint a;\nsca...,0,0,type mismatch
885,최수웅,21.07.31 17:23,원의넓이구하기,WA,0,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,0,0,type mismatch
890,강수인,21.03.07 11:21,원의넓이구하기,WA,2,#include<cstdio>\nint main(){\n double r;\n...,0,2,type mismatch
891,김가영,21.05.23 18:20,원의넓이구하기,WA,0,#include <cstdio>\nint main()\n{\n int a;\n...,0,0,type mismatch
892,김가영,21.05.23 18:20,원의넓이구하기,WA,0,#include <cstdio>\nint main()\n{\n int a;\n...,0,0,type mismatch
893,김가영,21.05.23 18:19,원의넓이구하기,WA,0,#include <cstdio>\nint main()\n{\n int a;\n...,"0,0",1,type mismatch


음수는0으로

In [ ]:
"음수는0으로" : {
    '0' : 'Output format mismatch',
    '1' : 'Incorrect calculation',
    '2' : 'Incorrect calculation',
    '3' : 'type mismatch',
    '4' : 'Incorrect calculation'
}

In [140]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Incorrect calculation
2 : Incorrect calculation
3 : type mismatch
4 : Incorrect calculation
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "음수는0으로") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]].to_string(index=False)

query1_22 = question_query  + result_query + context_query

ans1_22 = question(query1_22)
print(ans1_22)

review = [
        "Output format mismatch",
        "Output format mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Incorrect calculation",
        "Incorrect calculation",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Incorrect calculation",
        "Misunderstanding of the problem",
        "Output 

In [177]:
review = [
        "Output format mismatch",
        "Output format mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Incorrect calculation",
        "Incorrect calculation",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Incorrect calculation",
        "Misunderstanding of the problem",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch"
]

ce_rows = result_df[(result_df["문제 이름"] == "음수는0으로") & (result_df["정답 여부"] == "WA")]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "음수는0으로") & (result_df["정답 여부"] == "WA")]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
945,김정찬,21.10.23 21:44,음수는0으로,WA,0,#include<cstdio>\n\nint main()\n{\nint a;\nsca...,1,0,Output format mismatch
947,김정찬,21.10.23 21:44,음수는0으로,WA,0,#include<cstdio>\n\nint main()\n{\nint a;\nsca...,1,0,Output format mismatch
954,박유빈 yoobinsik,22.11.05 13:31,음수는0으로,WA,2,#include <cstdio>\n\nint main()\n{\n int a;...,0,2,Incorrect calculation
961,송지훈,21.08.30 21:17,음수는0으로,WA,4,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,1,1,Incorrect calculation
962,송지훈,21.08.29 21:25,음수는0으로,WA,0,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,1,0,Output format mismatch
963,신승명 himond1126,22.07.23 17:54,음수는0으로,WA,0,#include<cstdio>\n\nint main()\n{\n double ...,1,3,Output format mismatch
974,이진아,21.04.02 16:17,음수는0으로,WA,0,#include <cstdio>\nint main()\n{\nint a;\nscan...,1,0,Output format mismatch
976,이진아,21.04.17 13:29,음수는0으로,WA,0,#include <cstdio>\nint main()\n{\nint a;\nscan...,1,0,Output format mismatch
979,정시원,22.01.09 11:17,음수는0으로,WA,0,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,1,0,Output format mismatch
986,홍지형 aaron_moon,22.04.03 20:53,음수는0으로,WA,4,#include <cstdio>\n\nint main()\n{\n int a;...,1,4,Incorrect calculation


정수한개입력받아부호바꿔출력하기

In [ ]:
"정수한개입력받아부호바꿔출력하기" : {
    '0' : 'Incorrect calculation',
    '1' : 'Output format mismatch',
    '2' : 'Input format mismatch'
}

In [142]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Incorrect calculation
1 : Output format mismatch
2 : Input format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "정수한개입력받아부호바꿔출력하기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]].to_string(index=False)

query1_23 = question_query  + result_query + context_query

ans1_23 = question(query1_23)
print(ans1_23)

review = [
        "Input format mismatch",
        "Input format mismatch",
        "Input format mismatch",
        "Input format mismatch",
        "Input format mismatch",
        "Input format mismatch",
        "Output format mismatch",
        "Misunderstanding of the problem",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Input format mismatch",
        "Input format mismatch",
        "Input format mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Input format mismatch",
        "Incorrect calculation",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Input format mismatch",
        "Input format mismatch"
        ]


In [178]:
review = [
        "Input format mismatch",
        "Input format mismatch",
        "Input format mismatch",
        "Input format mismatch",
        "Input format mismatch",
        "Input format mismatch",
        "Output format mismatch",
        "Misunderstanding of the problem",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Input format mismatch",
        "Input format mismatch",
        "Input format mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "Incorrect calculation",
        "Input format mismatch",
        "Incorrect calculation",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Misunderstanding of the problem",
        "Input format mismatch",
        "Input format mismatch"
        ]

ce_rows = result_df[(result_df["문제 이름"] == "정수한개입력받아부호바꿔출력하기") & (result_df["정답 여부"] == "WA")]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "정수한개입력받아부호바꿔출력하기") & (result_df["정답 여부"] == "WA")]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
1033,김동우 derek0602,22.10.22 00:26,정수한개입력받아부호바꿔출력하기,WA,2,"#include<cstdio>\nint main(){\n int a,b;\n ...",0,2,Input format mismatch
1034,김동우 derek0602,22.10.22 10:41,정수한개입력받아부호바꿔출력하기,WA,2,"#include<cstdio>\nint main(){\n int a,b;\n ...",0,2,Input format mismatch
1035,김동우 derek0602,22.10.22 10:41,정수한개입력받아부호바꿔출력하기,WA,2,"#include<cstdio>\nint main(){\n int a,b;\n ...",0,2,Input format mismatch
1040,박유빈 yoobinsik,22.10.22 16:59,정수한개입력받아부호바꿔출력하기,WA,2,"#include<cstdio>\n\nint main()\n{\n int a,b...",0,2,Input format mismatch
1047,신승명 himond1126,22.07.16 16:29,정수한개입력받아부호바꿔출력하기,WA,2,#include<cstdio>\n\nint main()\n{\n int a=7...,0,2,Input format mismatch
1048,신승명 himond1126,22.07.16 16:30,정수한개입력받아부호바꿔출력하기,WA,2,#include<cstdio>\n\nint main()\n{\n int a=5...,0,2,Input format mismatch
1049,신승명 himond1126,22.07.16 16:31,정수한개입력받아부호바꿔출력하기,WA,1,#include<cstdio>\n\nint main()\n{\n int a=-...,0,2,Output format mismatch
1053,신제민,22.03.06 13:48,정수한개입력받아부호바꿔출력하기,WA,NaN,"#include<cstdio>\nint main()\n{\nint a,b;\nsca...",NOT,0,Misunderstanding of the problem
1054,신제민,22.03.06 13:46,정수한개입력받아부호바꿔출력하기,WA,0,#include<cstdio>\nint main()\n{\nint a;\nint b...,2,0,Incorrect calculation
1055,신제민,22.03.06 13:42,정수한개입력받아부호바꿔출력하기,WA,0,#include<cstdio>\nint main()\n{\nint a;\nint a...,2,0,Incorrect calculation


초를분,초단위로변환하기

In [ ]:
"초를분,초단위로변환하기" : {
    '0' : 'Output format mismatch',
    '1' : 'Output format mismatch',
    '2' : 'Output format mismatch',
    '3' : 'Incorrect calculation',
    '4' : 'Input format mismatch'
}

In [144]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Output format mismatch
2 : Output format mismatch
3 : Incorrect calculation
4 : Input format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "초를분,초단위로변환하기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][:20].to_string(index=False)

query1_24 = question_query  + result_query + context_query

ans1_24 = question(query1_24)
print(ans1_24)

Based on the given DataFrame, the error descriptions for each code are as follows:

review = [
    "Incorrect calculation",  # code 3
    "Input format mismatch",  # code 4
    "Incorrect calculation",  # code 3
    "Input format mismatch",  # code 4
    "Misunderstanding of the problem",  # NaN
    "Misunderstanding of the problem",  # NaN
    "Incorrect calculation",  # code 3
    "Output format mismatch",  # code 0
    "Output format mismatch",  # code 0
    "Output format mismatch",  # code 0
    "Output format mismatch",  # code 0
    "Output format mismatch",  # code 0
    "Output format mismatch",  # code 0
    "Incorrect calculation",  # code 3
    "Output format mismatch",  # code 0
    "Incorrect calculation",  # code 3
    "Misunderstanding of the problem",  # NaN
    "Output format mismatch",  # code 0
    "Output format mismatch",  # code 0
    "Output format mismatch"  # code 0
]


In [179]:

review = [
    "Incorrect calculation",  # code 3
    "Input format mismatch",  # code 4
    "Incorrect calculation",  # code 3
    "Input format mismatch",  # code 4
    "Misunderstanding of the problem",  # NaN
    "Misunderstanding of the problem",  # NaN
    "Incorrect calculation",  # code 3
    "Output format mismatch",  # code 0
    "Output format mismatch",  # code 0
    "Output format mismatch",  # code 0
    "Output format mismatch",  # code 0
    "Output format mismatch",  # code 0
    "Output format mismatch",  # code 0
    "Incorrect calculation",  # code 3
    "Output format mismatch",  # code 0
    "Incorrect calculation",  # code 3
    "Misunderstanding of the problem",  # NaN
    "Output format mismatch",  # code 0
    "Output format mismatch",  # code 0
    "Output format mismatch"  # code 0
]

ce_rows = result_df[(result_df["문제 이름"] == "초를분,초단위로변환하기") & (result_df["정답 여부"] == "WA")][:20]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "초를분,초단위로변환하기") & (result_df["정답 여부"] == "WA")][:20]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
1112,김동우 derek0602,22.10.22 11:03,"초를분,초단위로변환하기",WA,3,"#include<cstdio>\nint main(){\n int a,b;\n ...","2,0",4,Incorrect calculation
1113,김동우 derek0602,22.10.22 11:03,"초를분,초단위로변환하기",WA,4,"#include<cstdio>\nint main(){\n int a,b;\n ...","2,0",4,Input format mismatch
1115,김동우 derek0602,22.10.22 11:05,"초를분,초단위로변환하기",WA,3,"#include<cstdio>\nint main(){\n int a,b;\n ...","2,0",4,Incorrect calculation
1116,김동우 derek0602,22.10.22 11:05,"초를분,초단위로변환하기",WA,4,"#include<cstdio>\nint main(){\n int a,b;\n ...","2,0",4,Input format mismatch
1120,김정찬,21.10.17 17:49,"초를분,초단위로변환하기",WA,NaN,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,NOT,1,Misunderstanding of the problem
1121,김정찬,21.10.17 17:49,"초를분,초단위로변환하기",WA,NaN,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,NOT,1,Misunderstanding of the problem
1124,박윤선,21.07.24 21:18,"초를분,초단위로변환하기",WA,3,"#include<cstdio>\nint main(){\nint a;\nscanf(""...",2,3,Incorrect calculation
1125,박재윤,21.01.05 23:56,"초를분,초단위로변환하기",WA,0,#include <cstdio>\n\nint main()\n{\nint a; \ns...,0,0,Output format mismatch
1126,박재윤,21.01.07 00:56,"초를분,초단위로변환하기",WA,0,#include <cstdio>\n\nint main()\n{\nint a; \ns...,0,0,Output format mismatch
1127,박재윤,21.01.07 00:56,"초를분,초단위로변환하기",WA,0,#include <cstdio>\n\nint main()\n{\nint a; \ns...,0,0,Output format mismatch


In [146]:
question_query = f"""
1. Question :
The following Dataframe represents the programming code of students issued by Wrong Answer(WA).
Your task is to map the error classifications listed in the table above to the corresponding error description based on the following rules:

0 : Output format mismatch
1 : Output format mismatch
2 : Output format mismatch
3 : Incorrect calculation
4 : Input format mismatch
NaN : misunderstanding of the problem
-1 : misunderstanding of the problem

"""

context_query = "The DataFrame contains the following data.\n\n"  + df.loc[(df["문제 이름"] == "초를분,초단위로변환하기") & (df["정답 여부"] == "WA"), ["문제 이름", "오류 분류"]][20:].to_string(index=False)

query1_25 = question_query  + result_query + context_query

ans1_25 = question(query1_25)
print(ans1_25)

Here is the mapping of the error classifications to the corresponding error descriptions:

review = [
        "misunderstanding of the problem",
        "Incorrect calculation",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "Input format mismatch",
        "Incorrect calculation",
        "Incor

In [180]:
review = [
        "misunderstanding of the problem",
        "Incorrect calculation",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Incorrect calculation",
        "Output format mismatch",
        "Output format mismatch",
        "Output format mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "Input format mismatch",
        "Incorrect calculation",
        "Incorrect calculation",
        "Input format mismatch"
        ]

ce_rows = result_df[(result_df["문제 이름"] == "초를분,초단위로변환하기") & (result_df["정답 여부"] == "WA")][20:]

# 선택된 행들의 인덱스를 가져옴
ce_row_indices = ce_rows.index

# 상위 10개 행의 'review' 컬럼 값을 Error 리스트 값으로 변경
result_df.loc[ce_row_indices, 'review'] = review

result_df[(result_df["문제 이름"] == "초를분,초단위로변환하기") & (result_df["정답 여부"] == "WA")][20:]

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
1155,최도윤,21.10.11 09:17,"초를분,초단위로변환하기",WA,NaN,"#include <cstdio>\n\nint main() {\nint a,b,c;\...",NOT,1,misunderstanding of the problem
1156,최수웅,21.07.31 16:43,"초를분,초단위로변환하기",WA,3,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,2,3,Incorrect calculation
1171,강석준,21.06.27 17:51,"초를분,초단위로변환하기",WA,2,#include <cstdio>\n\nint main()\n{\n int a;...,0,2,Output format mismatch
1173,강석준,21.05.30 15:58,"초를분,초단위로변환하기",WA,2,#include <cstdio>\n\nint main()\n{\n int a;...,0,2,Output format mismatch
1177,김무영,21.03.07 11:59,"초를분,초단위로변환하기",WA,2,#include <cstdio>\n\n int main()\n{\n in...,0,1,Output format mismatch
1179,김무영,21.03.07 11:59,"초를분,초단위로변환하기",WA,2,#include <cstdio>\n\n int main()\n{\n in...,0,1,Output format mismatch
1180,김무진,21.05.27 17:03,"초를분,초단위로변환하기",WA,2,"#include<cstdio>\n\nint main()\n{\n int a, ...","0,2",2,Output format mismatch
1181,김무진,21.05.27 17:03,"초를분,초단위로변환하기",WA,3,"#include<cstdio>\n\nint main()\n{\n int a, ...","0,2",2,Incorrect calculation
1182,김무진,21.05.27 17:06,"초를분,초단위로변환하기",WA,2,"#include<cstdio>\n\nint main()\n{\n int a, ...","0,2",2,Output format mismatch
1183,김무진,21.05.27 17:06,"초를분,초단위로변환하기",WA,3,"#include<cstdio>\n\nint main()\n{\n int a, ...","0,2",2,Incorrect calculation


In [181]:
result_df.loc[ (result_df["정답 여부"] == "WA") & (result_df['review'].isnull())] 

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review


In [182]:
result_df.loc[ (result_df["정답 여부"] == "WA")] 

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
19,유재학 yujaehak16,22.08.07 18:16,10보다작을까?-1,WA,2,#include <cstdio>\nint main()\n{\n int a;\n...,1,2,off-by-one
20,유재학 yujaehak16,22.08.07 18:16,10보다작을까?-1,WA,2,#include <cstdio>\nint main()\n{\n int a;\n...,1,2,off-by-one
22,유제연,22.02.12 19:45,10보다작을까?-1,WA,NaN,#include<cstdio>\nint main()\n{\nint a;\nif(10...,NOT,1,misunderstanding of the problem
26,이연준,20.11.01 18:28,10보다작을까?-1,WA,1,#include <cstdio>\n\nint main()\n{\nint a;\nsc...,1,1,semantic error
32,전현준,22.03.11 17:21,10보다작을까?-1,WA,3,#include <cstdio>\n\nint main()\n{\nint a=11;\...,0,0,Output format mismatch
...,...,...,...,...,...,...,...,...,...
1464,허윤구,21.03.07 11:06,사칙연산,WA,2,"#include<cstdio>\nint main()\n{\n int a, b;...","0,0",0,Output format mismatch
1466,허윤구,21.03.07 11:11,사칙연산,WA,-1,"#include <cstdio>\nint main(){\n int a, b;\...",NOT,2,Misunderstanding of the problem
1467,허윤구,21.03.07 11:10,사칙연산,WA,-1,"#include <cstdio>\nint main()\n{\n int a, b...",NOT,4,Misunderstanding of the problem
1468,허윤구,21.03.07 11:07,사칙연산,WA,0,"#include<cstdio>\nint main()\n{\n int a, b;...","0,0",0,Output format mismatch


In [183]:
result_df.loc[ result_df["정답 여부"] =="WA" ,'대분류'] = result_df.loc[result_df["정답 여부"] =="WA", 'review']
result_df

,학생명,제출 시간,문제 이름,정답 여부,오류 분류,코드,대분류,prediction,review
0,강지민 jiminkang1404,22.11.01 22:08,10보다작을까?-1,AC,NaN,#include <cstdio>\nint main(){\n int a;\n ...,NOT,1,NaN
1,권현욱,21.12.26 10:24,10보다작을까?-1,AC,NaN,#include<stdio.h>\nint main()\n{\nint a;\nscan...,NOT,1,NaN
2,김강범 kkb110513,22.07.30 18:11,10보다작을까?-1,AC,NaN,#include <cstdio>\n\nint main()\n{\n int a;...,NOT,1,NaN
3,김동우 derek0602,22.11.05 10:41,10보다작을까?-1,AC,NaN,#include<cstdio>\nint main(){\n int a;\n ...,NOT,1,NaN
4,김민하,21.12.18 11:24,10보다작을까?-1,AC,NaN,"#include<cstdio>\nint main(){\nint a;\nscanf(""...",NOT,1,NaN
...,...,...,...,...,...,...,...,...,...
1466,허윤구,21.03.07 11:11,사칙연산,WA,-1,"#include <cstdio>\nint main(){\n int a, b;\...",Misunderstanding of the problem,2,Misunderstanding of the problem
1467,허윤구,21.03.07 11:10,사칙연산,WA,-1,"#include <cstdio>\nint main()\n{\n int a, b...",Misunderstanding of the problem,4,Misunderstanding of the problem
1468,허윤구,21.03.07 11:07,사칙연산,WA,0,"#include<cstdio>\nint main()\n{\n int a, b;...",Output format mismatch,0,Output format mismatch
1469,허윤구,21.03.07 11:07,사칙연산,WA,2,"#include<cstdio>\nint main()\n{\n int a, b;...",Output format mismatch,0,Output format mismatch


In [187]:
result_df.loc[result_df["정답 여부"] =="WA" ,'대분류'] = result_df.loc[result_df["정답 여부"] =="WA" ,'대분류'].str.lower()

unique_values = result_df['대분류'].unique()
print(unique_values)

['NOT' 'off-by-one' 'misunderstanding of the problem' 'semantic error'
 'output format mismatch' 'input format mismatch' 'incorrect calculation'
 'type mismatch']


In [189]:
result_df.to_excel('2차결과.xlsx', index=False, engine='openpyxl')